# This code trains the the model

In [2]:
import torch
import os
import numpy as np
from utils.datasets import get_hash

# path_to_data ="../data/dataset_transformed/drone/" 
path_to_data = "../data/real_dataset/drone/"

## Create cache file

In [3]:
# Load the data
cache_name = ["train", "test", "validation"]
for name in cache_name:
    with open(path_to_data + name + ".txt") as t:
        images = t.readlines()
    images = [x.strip("\n") for x in images]

    cache_dic = {}
    for img in images:
        label_list = np.array([], ndmin=2)
        data = list()
        with open(path_to_data + "labels/" + img[:-3] + "txt") as f:
            for line in f:
                label_list = np.hstack((label_list, np.array([float(x) for x in line.split(" ")], ndmin=2, dtype=np.float32)))

        data.append(label_list)
        data.append(path_to_data + "mask/" + img[:-3] + "jpg")
        data.append((640, 480))
        cache_dic[path_to_data + "images/" + img] = data


    cache_dic["hash"] = get_hash([path_to_data + "labels/" + x[:-3] + "txt" for x in images] + [path_to_data + "mask/" + x[:-3] + "png" for x in images] + [path_to_data + "images/" + x for x in images])
    print(cache_dic["hash"])
    cache_dic["results"] = (len(images), 0, 0, 0, 0, len(images), 0, 0, len(images))
    cache_dic["version"] = 0.1

    torch.save(cache_dic, path_to_data + name + ".cache")



2298500301
287491102
287373623


## Test of cv2 reading mask

In [9]:
import cv2
path = "../data/dataset_transformed/drone/mask/000000.jpg"

cv2.imread(path)

array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8)

In [21]:
# print my cache
cache_dic = torch.load("../data/dataset_transformed/drone/train.cache")
print(cache_dic)

{'../data/dataset_transformed/drone/images/000046.jpg': [array([[          0,     0.20464,     0.77645,     0.15669,     0.75354,     0.20128,      0.7464,      0.2066,     0.72006,     0.24623,     0.71342,     0.16856,     0.84804,     0.21325,     0.84213,     0.21697,     0.80303,     0.25667,     0.79735,     0.09998,     0.13461,         320,         320,
                640,         480,         320,         240,         640,         480]]), '../data/dataset_transformed/drone/mask/0046.pngr', (640, 480)], '../data/dataset_transformed/drone/images/000097.jpg': [array([[          0,     0.20574,     0.15233,     0.21741,     0.17168,     0.16705,     0.14723,     0.21703,     0.21436,      0.1716,      0.1939,     0.23027,     0.10214,     0.18187,    0.075602,     0.22872,     0.15017,     0.18489,     0.12791,    0.063215,     0.13876,         320,         320,
                640,         480,         320,         240,         640,         480]]), '../data/dataset_transformed/d

In [5]:
import torch

cache = torch.load(r"../data/LINEMOD/cat/train.cache")
print(cache)


FileNotFoundError: [Errno 2] No such file or directory: '../data/LINEMOD/cat/train.cache'

## transform dataset 



In [10]:
# Be sure to convert the data to our format focusing on the camera intrinsics
!python data_curation/convert_drone_dataset.py --path_to_dataset ../data/dataset_transformed

../data/dataset_transformed/drone/test.txt
../data/dataset_transformed/drone/train.txt
../data/dataset_transformed/drone/training_range.txt
../data/dataset_transformed/drone/validation.txt
../data/dataset_transformed/drone/test.cache
Traceback (most recent call last):
  File "/home/ondra/BC/yolov5/YOLOv5-6D-Pose-fork/data_curation/convert_drone_dataset.py", line 59, in <module>
    fix_image_paths(args.path_to_dataset)
  File "/home/ondra/BC/yolov5/YOLOv5-6D-Pose-fork/data_curation/convert_drone_dataset.py", line 37, in fix_image_paths
    lines = f.readlines()
  File "/home/ondra/anaconda3/envs/yolov5/lib/python3.9/codecs.py", line 322, in decode
    (result, consumed) = self._buffer_decode(data, self.errors, final)
UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 64: invalid start byte


## Train

In [5]:
# By default, the training script will download the COCO pretrained YOLOv5-v7 model. If this is no longer supported, you can download the weights from the YOLOv5-6D google drive (https://drive.google.com/drive/folders/11BW41xO3R1UBnc2Dx1xA3CPbYPGTrfHQ?usp=drive_link) and place them in the weights folder
!python train.py --batch 2 --epochs 1 --cfg models/yolov5s_6dpose_bifpn.yaml --hyp configs/hyp.single.yaml --weights ../data/weights/yolov5s.pt --data configs/linemod/cat.yaml --rect --optimizer Adam 

YOLOv5 77d2023 torch 1.9.1+cu102 CPU

Namespace(weights='../data/weights/yolov5s.pt', cfg='models/yolov5s_6dpose_bifpn.yaml', data='configs/linemod/cat.yaml', hyp='configs/hyp.single.yaml', epochs=1, batch_size=2, img_size=[640, 640], rect=True, resume=False, nosave=False, notest=False, noautoanchor=False, evolve=False, bucket='', cache_images=False, image_weights=False, device='', single_cls=False, optimizer='Adam', sync_bn=False, local_rank=-1, log_imgs=8, log_artifacts=False, workers=8, project='runs/train', entity=None, name='exp', exist_ok=False, quad=False, linear_lr=False, standard_lr=False, symetric=False, world_size=1, global_rank=-1, save_dir='runs/train/exp40', total_batch_size=2)
Start Tensorboard with "tensorboard --logdir runs/train", view at http://localhost:6006/
hyperparameters: lr0=0.0001, lrf=0.01, lr_factor=0.5, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=1.5, obj=0.1, obj_pw=1.0, cls=0.5, cls_pw=1.0, anchor_t

In [1]:
# By default, the training script will download the COCO pretrained YOLOv5-v7 model. If this is no longer supported, you can download the weights from the YOLOv5-6D google drive (https://drive.google.com/drive/folders/11BW41xO3R1UBnc2Dx1xA3CPbYPGTrfHQ?usp=drive_link) and place them in the weights folder
!python train.py --batch 2 --epochs 2 --cfg models/yolov5s_6dpose_bifpn.yaml --hyp configs/hyp.single.yaml --weights ../data/weights/yolov5s.pt --data configs/drone_data/drone.yaml --rect --optimizer Adam 

YOLOv5 211f7cb torch 1.9.1+cu102 CPU

Namespace(weights='../data/weights/yolov5s.pt', cfg='models/yolov5s_6dpose_bifpn.yaml', data='configs/drone_data/drone.yaml', hyp='configs/hyp.single.yaml', epochs=2, batch_size=2, img_size=[640, 640], rect=True, resume=False, nosave=False, notest=False, noautoanchor=False, evolve=False, bucket='', cache_images=False, image_weights=False, device='', single_cls=False, optimizer='Adam', sync_bn=False, local_rank=-1, log_imgs=8, log_artifacts=False, workers=8, project='runs/train', entity=None, name='exp', exist_ok=False, quad=False, linear_lr=False, standard_lr=False, symetric=False, world_size=1, global_rank=-1, save_dir='runs/train/exp8', total_batch_size=2)
Start Tensorboard with "tensorboard --logdir runs/train", view at http://localhost:6006/
hyperparameters: lr0=0.0001, lrf=0.01, lr_factor=0.5, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=1.5, obj=0.1, obj_pw=1.0, cls=0.5, cls_pw=1.0, anch

In [14]:
!python train.py --resume --exist-ok --name exp2

Resuming training from ./runs/train/exp4/weights/last.pt
YOLOv5 540623e torch 1.9.1+cu102 CPU

Namespace(weights='./runs/train/exp4/weights/last.pt', cfg='', data='configs/drone_data/drone.yaml', hyp='configs/hyp.single.yaml', epochs=4, batch_size=2, img_size=[640, 640], rect=True, resume=True, nosave=False, notest=False, noautoanchor=False, evolve=False, bucket='', cache_images=False, image_weights=False, device='', single_cls=False, optimizer='Adam', sync_bn=False, local_rank=-1, log_imgs=8, log_artifacts=False, workers=8, project='runs/train', entity=None, name='exp', exist_ok=False, quad=False, linear_lr=False, standard_lr=False, symetric=False, world_size=1, global_rank=-1, save_dir='runs/train/exp4', total_batch_size=2)
Start Tensorboard with "tensorboard --logdir runs/train", view at http://localhost:6006/
hyperparameters: lr0=0.0001, lrf=0.01, lr_factor=0.5, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=1.5, obj=0.1, obj_pw

### This is how to play the training with sudo

In [1]:
!sudo env "PATH=$PATH" python train.py --batch 4 --epochs 100 --cfg models/yolov5s_6dpose_bifpn.yaml --hyp configs/hyp.single.yaml --weights ../data/weights/yolov5s.pt --data configs/drone_data/drone.yaml --rect --optimizer Adam 

[sudo] password for vitaondr: 
^C


## Real dataset train

In [6]:

!python train.py --batch 2 --epochs 1 --cfg models/yolov5s_6dpose_bifpn.yaml --hyp configs/hyp.single.yaml --weights ../data/weights/yolov5s.pt --data configs/drone_data/drone_real.yaml --rect --optimizer Adam #--img-size 1280

YOLOv5 3a745a6 torch 1.9.1+cu102 CPU

Namespace(weights='../data/weights/yolov5s.pt', cfg='models/yolov5s_6dpose_bifpn.yaml', data='configs/drone_data/drone_real.yaml', hyp='configs/hyp.single.yaml', epochs=1, batch_size=2, img_size=[640, 640], rect=True, resume=False, nosave=False, notest=False, noautoanchor=False, evolve=False, bucket='', cache_images=False, image_weights=False, device='', single_cls=False, optimizer='Adam', sync_bn=False, local_rank=-1, log_imgs=8, log_artifacts=False, workers=8, project='runs/train', entity=None, name='exp', exist_ok=False, quad=False, linear_lr=False, standard_lr=False, symetric=False, world_size=1, global_rank=-1, save_dir='runs/train/exp22', total_batch_size=2)
Start Tensorboard with "tensorboard --logdir runs/train", view at http://localhost:6006/
hyperparameters: lr0=0.0001, lrf=0.01, lr_factor=0.5, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=1.5, obj=0.1, obj_pw=1.0, cls=0.5, cls_pw=1.0

## Inference

In [14]:
!python detect.py --weights ../data/LINEMOD/cat/weights/best.pt --img 640 --conf 0.25 --source ../data/LINEMOD/cat/JPEGImages/000000.jpg --static-camera configs/linemod/linemod_camera.json --mesh-data ../data/LINEMOD/cat/cat.ply

Namespace(weights=['../data/LINEMOD/cat/weights/best.pt'], source='../data/LINEMOD/cat/JPEGImages/000000.jpg', static_camera='configs/linemod/linemod_camera.json', mesh_data='../data/LINEMOD/cat/cat.ply', img_size=640, conf_thres=0.25, device='', view_img=False, name='exp', exist_ok=False, project='runs/detect')
YOLOv5 3a745a6 torch 1.9.1+cu102 CPU

Fusing layers... 
Model Summary: 391 layers, 97574820 parameters, 0 gradients, 225.4 GFLOPS
[[  -0.033737   -0.033737   -0.033737   -0.033737    0.033274    0.033274    0.033274    0.033274]
 [  -0.061801   -0.061801    0.065832    0.065832   -0.061801   -0.061801    0.065832    0.065832]
 [  -0.045446    0.072011   -0.045446    0.072011   -0.045446    0.072011   -0.045446    0.072011]
 [          1           1           1           1           1           1           1           1]]
image 1/1 /home/ondra/BC/yolov5/YOLOv5-6D-Pose-fork/../data/LINEMOD/cat/JPEGImages/000000.jpg: tensor([[364.15668, 161.39651, 332.66818, 173.51559, 323.54919, 

In [11]:
!python detect.py --weights runs/train/exp19/weights/best.pt --img 640 --conf 0.25 --source /home/ondra/BC/MRS_simulation/mrs-flight-forge-connector-fork/dataset_transformed/drone/images/000179.jpg --static-camera configs/drone_data/drone_camera.json --mesh-data ../data/dataset_transformed/drone/drone.ply

Namespace(weights=['runs/train/exp19/weights/best.pt'], source='/home/ondra/BC/MRS_simulation/mrs-flight-forge-connector-fork/dataset_transformed/drone/images/000179.jpg', static_camera='configs/drone_data/drone_camera.json', mesh_data='../data/dataset_transformed/drone/drone.ply', img_size=640, conf_thres=0.25, device='', view_img=False, name='exp', exist_ok=False, project='runs/detect')
YOLOv5 3a745a6 torch 1.9.1+cu102 CPU

Fusing layers... 
Model Summary: 226 layers, 8894676 parameters, 0 gradients, 19.4 GFLOPS
image 1/1 /home/ondra/BC/MRS_simulation/mrs-flight-forge-connector-fork/dataset_transformed/drone/images/000179.jpg: tensor([[150.47800, 315.88943, 165.93604, 341.61694, 139.29539, 327.27637, 136.67024, 342.94180, 110.52475, 330.38013, 185.84938, 298.49649, 161.24179, 281.97772, 155.70067, 303.04956, 131.86714, 288.33240,   0.89323,   0.00000]])
480x640 1 drone, Done. (0.091s)
Results saved to runs/detect/exp15
1 labels saved to runs/detect/exp15/labels
Done. (0.558s)


## Test

In [29]:
!python test.py --batch 1 --weights runs/train/train5k1kepoch/weights/best.pt --data configs/drone_data/drone.yaml --task test --save-txt --save-hybrid --test-plotting


Namespace(weights=['runs/train/train5k1kepoch/weights/best.pt'], data='configs/drone_data/drone.yaml', batch_size=1, img_size=640, conf_thres=0.01, num_keypoints=9, task='test', device='', single_cls=False, augment=False, verbose=False, save_txt=True, save_hybrid=True, save_conf=False, save_json=False, project='runs/test', name='exp', exist_ok=False, symetric=False, test_plotting=True)
YOLOv5 211f7cb torch 1.9.1+cu102 CPU

Fusing layers... 
Model Summary: 226 layers, 8894676 parameters, 0 gradients, 19.4 GFLOPS
test: Scanning '../data/dataset_transformed/drone/test.cache' for images, labels
  0%|                                                    | 0/40 [00:00<?, ?it/s]rvec, tvec control print:  [     1.8553      0.1353     0.67742] [    -2.4376      2.2882      5.5954]
/home/ondra/anaconda3/envs/yolov5/lib/python3.9/site-packages/torch/cuda/amp/autocast_mode.py:120: UserWarning: torch.cuda.amp.autocast only affects CUDA ops, but CUDA is not available.  Disabling.
  warnings.warn("torc

In [4]:
!python test.py --batch 1 --weights runs/train/train5k1kepoch/weights/best.pt --data configs/drone_data/drone_real.yaml --task test --save-txt

Namespace(weights=['runs/train/train5k1kepoch/weights/best.pt'], data='configs/drone_data/drone_real.yaml', batch_size=1, img_size=640, conf_thres=0.01, num_keypoints=9, task='test', device='', single_cls=False, augment=False, verbose=False, save_txt=True, save_hybrid=False, save_conf=False, save_json=False, project='runs/test', name='exp', exist_ok=False, symetric=False, test_plotting=False)
YOLOv5 211f7cb torch 1.9.1+cu102 CPU

Fusing layers... 
Model Summary: 226 layers, 8894676 parameters, 0 gradients, 19.4 GFLOPS
test: Scanning '../data/real_dataset/drone/test.cache' for images, labels, masks
  0%|                                                  | 0/2082 [00:00<?, ?it/s]rvec, tvec control print:  [    0.91458     -1.1406      1.1989] [     1.7763      1.4371      3.9347]
/home/ondra/anaconda3/envs/yolov5/lib/python3.9/site-packages/torch/cuda/amp/autocast_mode.py:120: UserWarning: torch.cuda.amp.autocast only affects CUDA ops, but CUDA is not available.  Disabling.
  warnings.war